## 상품정보 데이터

### json데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

import re
import warnings
import os
warnings.filterwarnings(action='ignore')
#pd.set_option('옵션 이름', 수정할 값)
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [ ]:
items = pd.read_json('./items.json')
items.head()



In [ ]:
items.info()

### 상품명 전처리

In [ ]:
def item_name(row):
    return row.split('\n')[0] # 줄바꿈이전 문자열만 남기기

items['name'] = items['name'].apply(item_name)
items.head()

### 상품명 중복확인

In [ ]:
dup_idx = items.loc[items['name'].duplicated()==True].index
items.drop(dup_idx, axis=0, inplace=True)
items['name'].duplicated().sum()


In [ ]:
# 인덱스 재설정
items.reset_index(drop=True, inplace=True)

In [ ]:
items.info()

### Item_id 부여

In [ ]:
items['item_id'] = [(5-len((str(i)))) * '0' + str(i) for i in list(items.index)]
items.head()

In [ ]:
items.tail()

In [ ]:
# items.to_json('../Load/items_prc.json', orient='records', indent=2, force_ascii=False) # 객체기준 레코드, 들여쓰기 적용, 한글깨짐 해결

In [ ]:
# # 리뷰 크롤링용 데이터 저장
# items[['item_id', 'link']].to_json('../Extract/items_link.json', orient='records', indent=2, force_ascii=False)

# review data

In [ ]:
review = pd.read_json('./reviews.json')

# 불필요한 컬럼 삭제
review = review[['품번', '아이디', '별점', '키', '몸무게', '평소사이즈', '타입', '리뷰', '사이즈', '컬러', '색상', '색상-사이즈', '타입2', '데님']]
review.info()


### ['사이즈', '컬러', '색상', '타입', '타입2', '색상-사이즈', '데님'] 컬럼 통일 필요 = **['사이즈: str', '색상: 한글']으로 통일**


<img src='./imgs/타입_해당컬럼.png' width=500 height=500/>

#### 각 컬럼별 카디널리티 줄이고 컬럼 통일

In [ ]:
# 카디널리티 확인
cols = ['사이즈', '컬러', '색상', '타입', '타입2', '색상-사이즈', '데님']
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

In [ ]:
# 사이즈 컬럼
def size(row):
    if row in ['1', '2', '3', '4', 'F', '블랙(black)-2', '블랙 1']:
        return row
    elif row in ['1size', '미니-1', '1사이즈']:
        return re.sub('[^0-9]', '', row)
    elif row=='free':
        return 'F'
review['사이즈'] = review['사이즈'].apply(size)

review.loc[review['사이즈']=='블랙 1'] = '블랙-1'

# 컬러, 색상 컬럼
def color(row):
    if type(row) != float:
        return row
    
review['컬러'] = review['컬러'].apply(color)
review['색상'] = review['색상'].apply(color)

# 타입 컬럼
def type_prc(row):
    if not row:
        pass
    elif '. ' in row: 
        return row.split('-')[-1]
    elif '[예약판매]' in row:
        row = row.replace('[예약판매]', '')
        return row
    elif '(반팔)' in row:
        row = row.replace('(반팔)', '')
        return row
    elif '_' in row:
        return row.split('_')[-1]
    elif '.' in row:
        return None
    elif type(row)==str:
        return row

review['타입'] = review['타입'].apply(type_prc)
    
# 데님, 색상-사이즈 컬럼
def denim_color(row):
    if type(row) == str:
        if '예약' in row:
            return row.split(' ')[0]
        else:
            return row

review['데님'] = review['데님'].apply(denim_color)
review['색상-사이즈'] = review['색상-사이즈'].apply(denim_color)
review['타입2'] = review['타입2'].apply(denim_color)

전: <img src='./imgs/카디널리티_전.png'> 후: <img src='./imgs/카디널리티_후.png'>

#### ['색상', '사이즈'] 컬럼으로 통일

In [ ]:
# 카디널리티 확인
cols = ['사이즈', '컬러', '색상', '타입', '타입2', '색상-사이즈', '데님']
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

In [ ]:
# 색상-사이즈 형태인 경우
    ## str.split()로 분리, 문자열 정규화로 데이터 수정
# 색상 데이터만 있는 경우
    ## 문자열 정규화만 수행
def col_merge(df, cols):
    def normalize(row):
        row[0] = re.sub('[^가-힣]','',row[0]) # 한글만 남기기
        return row

    # 사이즈 컬럼 별도 수행
    idx = df.loc[(review['사이즈']=='블랙-1') | (review['사이즈']=='블랙(black)-2')].index
    for i in idx:
        df.loc[i, ['색상', '사이즈']] = df.iloc[i][col].split('-')
    for col in cols:
        idx = df.loc[df[col].notnull()==True].index
        for i in idx:
            row = df.iloc[i][col].split('-')
            if len(row)==2:
                df.loc[i, ['색상', '사이즈']] = normalize(row)
            else:
                df.loc[i, ['색상']] = normalize(row)
    return df
cols = ['컬러', '타입', '타입2', '색상-사이즈', '데님', '색상']
review = col_merge(review, cols)

* 나머지 컬럼은 삭제

In [ ]:
# 통합 끝난 컬럼 제거 (색상, 사이즈)
review.drop(columns=cols[:-1], inplace=True)

* 카디널리티 확인

In [ ]:
cols = ['사이즈', '색상']
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

### '별점' 컬럼 핫핑 직원 데이터는 제외

In [ ]:
staffs = set()
p = re.compile('[ㄱ-힣]')
for name in review['아이디'].unique():
    r = p.search(name)
    if r:
        staffs.add(name)
          

In [36]:
staffs

{'블랙-1',
 '스탭리뷰',
 '카카오톡체크아웃',
 '카카오페이 구매',
 '핫핑 - 도경',
 '핫핑 - 수연',
 '핫핑-가나',
 '핫핑-가원',
 '핫핑-다슬',
 '핫핑-다혜',
 '핫핑-도티',
 '핫핑-모모',
 '핫핑-미영',
 '핫핑-민주',
 '핫핑-민지',
 '핫핑-빈이',
 '핫핑-상이',
 '핫핑-선미',
 '핫핑-소연',
 '핫핑-소정',
 '핫핑-소현',
 '핫핑-수린',
 '핫핑-수지',
 '핫핑-수진',
 '핫핑-숙련',
 '핫핑-시은',
 '핫핑-썬',
 '핫핑-아랑',
 '핫핑-안나',
 '핫핑-여민',
 '핫핑-연재',
 '핫핑-유경',
 '핫핑-유리',
 '핫핑-유림',
 '핫핑-유빈',
 '핫핑-유진',
 '핫핑-윤정',
 '핫핑-윤주',
 '핫핑-은지',
 '핫핑-이소',
 '핫핑-정',
 '핫핑-정유',
 '핫핑-정은',
 '핫핑-정인',
 '핫핑-조유',
 '핫핑-주은',
 '핫핑-지니',
 '핫핑-지민',
 '핫핑-지원',
 '핫핑-지혜',
 '핫핑-진',
 '핫핑-쭈',
 '핫핑-채은',
 '핫핑-초롱',
 '핫핑-하린',
 '핫핑-현서',
 '핫핑-현진',
 '핫핑-혜원',
 '핫핑-효진',
 '핫핑-희성',
 '핫핑_B.F',
 '핫핑_MJ',
 '핫핑_미영',
 '핫핑_현진',
 '핫핑스탭',
 '핫핑스텝'}